## File 08 - Two-part model (Logistic + Linear Regression)

##### Group 12:

##### Hannah Schmuckler, mmc4cv

##### Rob Schwartz, res7cd

In this file, we create a small ML pipeline based on the output from File 02 (Feature creation).

We create a two part model made up of a logistic and a linear regression.

### Set up Spark session

We can specify more options in the SparkSession creator, but currently the options are at the default settings.

In [1]:
%%time
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, LinearRegressionModel, LinearRegressionSummary
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col
from pyspark.sql.functions import log
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import when
from pyspark.ml.classification import LogisticRegression
from handyspark import *
from matplotlib import pyplot as plt
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import lit

import pandas as pd
import numpy as np
import copy

spark = SparkSession.builder \
        .appName("project") \
        .getOrCreate()

sc = spark.sparkContext

CPU times: user 983 ms, sys: 714 ms, total: 1.7 s
Wall time: 6.56 s


### Read in dataframes for train and test sets

This data should have been previously generated: we can find it in the `processed_data` folder.

In [2]:
%%time
trainDF = spark.read.parquet("./processed_data/train.parquet")
testDF = spark.read.parquet("./processed_data/test.parquet")
# trainDF.show(5)

CPU times: user 4.08 ms, sys: 1.84 ms, total: 5.92 ms
Wall time: 3.12 s


### Label trainDF with 0 if they did not spend in m2 and 1 if they did

In [3]:
trainDF = trainDF.withColumn('logistic_T_total_spend', when(trainDF.T_total_spend == 0, 0)
                                                       .otherwise(1))
testDF = testDF.withColumn('logistic_T_total_spend', when(testDF.T_total_spend == 0, 0)
                                                       .otherwise(1))

trainDF = trainDF.withColumn('logistic_T_total_spend', col("logistic_T_total_spend").cast(DoubleType()))
testDF = testDF.withColumn('logistic_T_total_spend', col("logistic_T_total_spend").cast(DoubleType()))

### Set up Spark ML pipeline training for logistic regression

Here we decide which input columns should be used in order to create our training pipeline. To implement this step, we create the function `generatePipeline(inputCols, outputCol)`. Then, we train the pipeline using this function.

In [40]:
%%time

def generateLogisticPipeline(inputCols, outputCol, threshold):
    # Select input columns for linear regression
    vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features")

    # Select output column for linear regression
    logR = LogisticRegression(featuresCol="features", labelCol=outputCol, rawPredictionCol = "Logistic_Probabilities", predictionCol = "Logistic_Predictions", threshold = threshold)

    # The following lines (pipeline creation and fitting) replace these two commented-out lines.
    # vecTrainDF = vecAssembler.transform(trainDF)
 
    pipeline = Pipeline(stages=[vecAssembler, logR])
    return pipeline
    
#pipeline = generateLogisticPipeline(inputCols, "logistic_T_total_spend")
#pipelineModel = pipeline.fit(trainDF)



CPU times: user 3 µs, sys: 3 µs, total: 6 µs
Wall time: 9.54 µs


### View the model information

Create function to print out the model coefficients for the overall model, RMSE and R^2. We define the functions `modelInfo(inputCols, pipelineModel)` and `getEvaluationMetrics(pipelineModel,outputCol,testDF)` to report this information.

In [41]:
def modelInfo(inputCols, pipelineModel):
    # Create a zipped list containing the coefficients and the data
    modelCols = copy.deepcopy(inputCols)
    modelCoeffs = list(pipelineModel.stages[-1].coefficients)
    modelCoeffs.insert(0,pipelineModel.stages[-1].intercept)
    modelCols.insert(0,"intercept")
    modelZippedList = list(map(list, zip(modelCols, modelCoeffs)))
    
    
    # Create the pandas DataFrame
    modelDF = pd.DataFrame(modelZippedList, columns = ['Column name', 'Coefficient'])
    #modelDF['pValues'] = pvals
    return modelDF

In [42]:
# Calculate adjusted r2 (https://towardsdatascience.com/machine-learning-linear-regression-using-pyspark-9d5d5c772b42)
# This function will allow us to calculate the adjusted r-square value when we do PCA later. The default r-square function does not take into account k. 
def adj_r2(r2, inputCols, testDF, k = 0):
    n = testDF.count()
    if k == 0:
        p = len(inputCols)
    else: 
        p = len(inputCols) + k - 1
    
    adjusted_r2 = 1-(((1-r2)*(n-1))/(n-p-1))
    return adjusted_r2

In [43]:
# https://towardsdatascience.com/binary-classifier-evaluation-made-easy-with-handyspark-3b1e69c12b4f
def getLogisticEvaluationMetrics(pipelineModel,outputCol,testDF,inputCols):
    predDF = pipelineModel.transform(testDF)
    preds_only = predDF.select(outputCol, "Logistic_Predictions")
    preds_only.show(5)
    
    preds_only = preds_only.rdd
    
    metrics = MulticlassMetrics(preds_only)
    print(metrics.confusionMatrix().toArray())
    print("Accuracy: " + str(metrics.accuracy))

    return predDF


## Logistic regression part 1

In [106]:
%%time
inputCols = ["total_spend", "total_events", "total_sessions", "avg_session_length", "sd_session_length", "avg_interactions_per_session", 
             "sd_interactions_per_session", "max_interactions_per_session", "purchase_pct_of_total_events", "view_pct_of_total_events",
             "cart_pct_of_total_events", "avg_purchases_per_session", "sd_purchases_per_session", "cart_events", "purchase_events", 
             "sessions_with_purchase", "sessions_with_cart", "sessions_with_view", "pct_sessions_end_purchase", 
             "pct_sessions_end_cart", "total_spend_log", "total_events_log", "purchase_events_log", "total_sessions_log",
             "avg_session_length_log"]
outputCol= "logistic_T_total_spend"

pipeline = generateLogisticPipeline(inputCols, outputCol, .2) #Iterative testing determined that a threshold of .2 led to the highest overal adj R^2 for the model
pipelineModel = pipeline.fit(trainDF)

print(modelInfo(inputCols, pipelineModel))

logistic_predictions = getLogisticEvaluationMetrics(pipelineModel,outputCol,testDF, inputCols)
#print(f"RMSE is {evaluationMetrics[0]:.1f}")
#print(f"R^2 is {evaluationMetrics[1]:.5f}")
#print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

                     Column name  Coefficient
0                      intercept    -1.087309
1                    total_spend     0.000013
2                   total_events     0.002369
3                 total_sessions     0.147414
4             avg_session_length    -0.000002
5              sd_session_length     0.000003
6   avg_interactions_per_session    -0.012783
7    sd_interactions_per_session     0.004939
8   max_interactions_per_session    -0.003006
9   purchase_pct_of_total_events    -2.195029
10      view_pct_of_total_events    -0.433863
11      cart_pct_of_total_events    -0.796743
12     avg_purchases_per_session    -0.272801
13      sd_purchases_per_session     0.165081
14                   cart_events    -0.005728
15               purchase_events    -0.043132
16        sessions_with_purchase     0.076717
17            sessions_with_cart    -0.018144
18            sessions_with_view    -0.162480
19     pct_sessions_end_purchase     1.411522
20         pct_sessions_end_cart  

#### Threshold of .2 maximizes model adj-r^2, though it does not maximize the accuracy of this logistic regression. 

#### Next, we'll train a linear regression model using only the true positive training data, and generate a regression prediction for all test data that had a prediction of positive. 

In [113]:
part_2_train = trainDF.filter(col("logistic_T_total_spend") == 1)
part_2_test = logistic_predictions.filter(col("Logistic_Predictions") == 1).withColumnRenamed('probability', "ProbabilityLogistic")

part_1_test = logistic_predictions.filter(col("Logistic_Predictions") == 0).withColumnRenamed('probability', "ProbabilityLogistic") \
    .withColumn("", lit(None)) ## For overall accuracy metrics later, will be joined with output of part_2_test's regression predictions
#part_1_test = part_1_test.withColumn("")

In [114]:
print(part_2_train.count())
print(part_2_test.count())

74751
36563


In [115]:
%%time

def generateLinearPipeline(inputCols, outputCol):
    # Select input columns for linear regression
    vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features2")

    # Select output column for linear regression
    lr = LinearRegression(featuresCol="features2", labelCol=outputCol)

    # The following lines (pipeline creation and fitting) replace these two commented-out lines.
    # vecTrainDF = vecAssembler.transform(trainDF)
 
    pipeline = Pipeline(stages=[vecAssembler, lr])
    return pipeline
    
pipeline = generateLinearPipeline(inputCols, "T_total_spend")
pipelineModel = pipeline.fit(trainDF)



CPU times: user 14.2 ms, sys: 7.07 ms, total: 21.2 ms
Wall time: 1.13 s


In [116]:
def getLinearEvaluationMetrics(linearPipelineModel,outputCol,testDF,inputCols):
    new_preds = pipelineModel.transform(testDF)
    #new_preds.show(1)
    #part_1_test.show(1)
    
    predDF = new_preds.unionByName(part_1_test, allowMissingColumns=True)
    #predDF.show(1)

    predDF = predDF.withColumn("FinalPredictions", when(col("Logistic_Predictions") == 0, 0.0)
                               .otherwise(col("prediction")))
    #predDF.select(outputCol, "prediction").show(10)

    predDF.select(outputCol, "FinalPredictions").show()
    
    regressionEvaluator = RegressionEvaluator(
    predictionCol="FinalPredictions",
    labelCol=outputCol,
    metricName="rmse")
    rmse = regressionEvaluator.evaluate(predDF)

    regressionEvaluator = RegressionEvaluator(
    predictionCol="FinalPredictions",
    labelCol=outputCol,
    metricName="r2")
    r2 = regressionEvaluator.evaluate(predDF)
      
    # Manually calculate Adjusted r2
    adjusted_r2 = adj_r2(r2, inputCols, predDF)
    
    return rmse, r2, adjusted_r2, predDF

In [119]:
print("** All original inputs, original output **")
inputCols = ["total_spend","total_events","purchase_events", "total_sessions", "avg_session_length", "avg_interactions_per_session", "max_interactions_per_session",
             "purchase_pct_of_total_events", "view_pct_of_total_events", "cart_pct_of_total_events","avg_purchases_per_session", "cart_events", "view_events", 
             "sessions_with_purchase", "sessions_with_cart","sessions_with_view", "pct_sessions_end_purchase", "pct_sessions_end_cart", 'sd_session_length', 
             'sd_interactions_per_session', 'sd_purchases_per_session']


outputCol = "T_total_spend"

pipeline = generateLinearPipeline(inputCols, outputCol)
pipelineModel = pipeline.fit(part_2_train)

print("Model coefficients")
print(modelInfo(inputCols, pipelineModel))

evaluationMetrics = getLinearEvaluationMetrics(pipelineModel,outputCol,part_2_test, inputCols)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

** All original inputs, original output **
Model coefficients
                     Column name  Coefficient
0                      intercept   759.390544
1                    total_spend     0.591320
2                   total_events    -0.515724
3                purchase_events    41.207359
4                 total_sessions   -63.135401
5             avg_session_length    -0.000520
6   avg_interactions_per_session   -16.487473
7   max_interactions_per_session     6.873946
8   purchase_pct_of_total_events   443.417367
9       view_pct_of_total_events  -103.816335
10      cart_pct_of_total_events    26.430407
11     avg_purchases_per_session  -337.477860
12                   cart_events     5.579578
13                   view_events    -1.099834
14        sessions_with_purchase  -193.609708
15            sessions_with_cart   148.251893
16            sessions_with_view    60.564478
17     pct_sessions_end_purchase   454.561538
18         pct_sessions_end_cart  -854.405260
19             sd_

In [127]:
%%time
print("** Champion Model **")
inputCols = ["total_spend","purchase_events","avg_interactions_per_session", 
             "purchase_pct_of_total_events","cart_events", "view_events", 
             "sessions_with_purchase", "sessions_with_cart","pct_sessions_end_purchase", "pct_sessions_end_cart",  
             'sd_purchases_per_session']

outputCol = "T_total_spend"
# Below creation of pipeline is necessary for crossval to run. I wonder if there's a way to get it to run on the generate pipeline fn? 
vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features")

# Select output column for linear regression
lr = LinearRegression(featuresCol="features", labelCol="T_total_spend")

pipeline = Pipeline(stages=[vecAssembler, lr])
    
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0, 0.01, 0.2, 1, 10]) \
    .addGrid(lr.elasticNetParam, [0, 0.25, 0.5, 0.75, 1]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator().setLabelCol("T_total_spend"),
                          numFolds=4)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.setParallelism(8).fit(part_2_train)

** Champion Model **
CPU times: user 5.29 s, sys: 1.78 s, total: 7.08 s
Wall time: 28.9 s


In [128]:
print("Best model coefficients")
print(modelInfo(inputCols, cvModel.bestModel))


evaluationMetrics = getLinearEvaluationMetrics(cvModel.bestModel,"T_total_spend",part_2_test,inputCols)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

print()
print(cvModel.getEstimatorParamMaps()[np.argmax(cvModel.avgMetrics)])

Best model coefficients
                     Column name  Coefficient
0                      intercept   701.437134
1                    total_spend     0.591951
2                purchase_events     2.768463
3   avg_interactions_per_session   -18.786724
4   purchase_pct_of_total_events   513.610323
5                    cart_events    14.224328
6                    view_events    -0.595592
7         sessions_with_purchase   -52.616400
8             sessions_with_cart    52.633232
9      pct_sessions_end_purchase    44.023084
10         pct_sessions_end_cart  -216.749932
11      sd_purchases_per_session   196.179439
+------------------+------------------+
|     T_total_spend|  FinalPredictions|
+------------------+------------------+
|               0.0| 624.7580419820692|
|               0.0| 526.3177997157919|
| 312.4200134277344| 557.3234961943676|
|  6009.78010559082| 1652.212288207612|
|1627.1900024414062| 854.8646851048385|
| 573.9199829101562| 782.6884607497873|
| 622.400024414062